<a href="https://colab.research.google.com/github/martinoscarrodriguez/Analisis_y_Visualizacion_TP/blob/main/Cintelink_AyVdD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis y Visualización de Datos 

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn
from datetime import datetime, timedelta

seaborn.set_context('talk')
# Set float format
pd.set_option('display.float_format','{:.2f}'.format)

# Set style
seaborn.set_style("darkgrid")
seaborn.set_palette('pastel')
seaborn.set_context("paper", rc={"font.size":12,"axes.titlesize":12,"axes.labelsize":12}) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def convert2float32(n):
    try:
        return numpy.float32(n)
    except:
        return numpy.nan

def convert2float16(n):
    try:
        return numpy.float16(n)
    except:
        return numpy.nan

def convert2int16(n):
    try:
        return numpy.int16(n)
    except:
        return numpy.nan

def convert2int8(n):
    try:
        return numpy.int8(n)
    except:
        return numpy.nan

In [ ]:
dtypes = {
    "id": "category",
    "id_equipo": "category",
    "id_tanque": "category",
    "producto": "category",
    "id_empresa": "category",
    "id_canal": "category",
    "nombre_producto": "category",
    "industria": "category",
    "alarma": "boolean"
}

# Para evitar datos erróneos en el parsing
converters = {
    "id_industria": convert2float16,
    "volumen": convert2float32,
    "vbat1": convert2int16,
    "vbat2": convert2int16,
    "capacidad": convert2float32,
    "fuel_level_dmm": convert2float32,
    "water_level_dmm": convert2float32,
    "water_volume_lts": convert2float32,
    "temp5": convert2float16,
    "temp4": convert2float16,
    "temp3": convert2float16,
    "temp2": convert2float16,
    "temp1": convert2float16,
    "temperatura": convert2float16,
    "coef_var_vol": convert2float16,
    "density": convert2float32
}

In [ ]:
# filename = '/content/drive/MyDrive/Colab Notebooks/DiploDatos/Mentoría/Datasets/dataset100mil.csv'
filename = '/content/drive/MyDrive/Colab Notebooks/DiploDatos/Mentoría/Datasets/StorageInventory_2021_Q1.csv'
raw_df = pd.read_csv(filename, converters=converters, dtype=dtypes, parse_dates=["timestamp"])

In [ ]:
raw_df.info(memory_usage="deep")

In [ ]:
raw_df.head()

## Limpieza de datos

### Tipo de datos de columnas

In [ ]:
raw_df['volumen'] = raw_df['volumen'].astype(numpy.float32)
raw_df['temperatura'] = raw_df['temperatura'].astype(numpy.float16)
raw_df['temp5'] = raw_df['temp5'].astype(numpy.float16)
raw_df['temp4'] = raw_df['temp4'].astype(numpy.float16)
raw_df['temp3'] = raw_df['temp3'].astype(numpy.float16)
raw_df['temp2'] = raw_df['temp2'].astype(numpy.float16)
raw_df['temp1'] = raw_df['temp1'].astype(numpy.float16)
raw_df['fuel_level_dmm'] = raw_df['fuel_level_dmm'].astype(numpy.float32)
raw_df['water_level_dmm'] = raw_df['water_level_dmm'].astype(numpy.float32)
raw_df['water_volume_lts'] = raw_df['water_volume_lts'].astype(numpy.float32)
raw_df['capacidad'] = raw_df['capacidad'].astype(numpy.float32)
raw_df['coef_var_vol'] = raw_df['coef_var_vol'].astype(numpy.float16)
raw_df['density'] = raw_df['density'].astype(numpy.float16)
raw_df['vbat1'] = raw_df['vbat1'].astype(numpy.int16)
raw_df['vbat2'] = raw_df['vbat2'].astype(numpy.int16)
raw_df['id_industria'] = raw_df['id_industria'].astype("category")
raw_df['codigo'] = raw_df['codigo'].astype("str")

In [ ]:
raw_df.info(memory_usage="deep")

In [ ]:
# Trabajamos sobre una copia y no sobre el original
df = raw_df.copy()

In [ ]:
# Si se llena la memoria, ejecutar
del raw_df

### Selección de columnas relevantes

### Valores faltantes

Las columnas `volumen` y `temperatura`

#### Volúmenes nulos

In [ ]:
nan = numpy.nan
df.query("volumen == @nan")

#### Temperaturas nulas

In [ ]:
nan = numpy.nan
df.query("temperatura == @nan")

#### Niveles de combustible, temperatura y códigos

In [ ]:
#análisis de intersección nan entre variables
df[df['temperatura'].isna()&df['codigo'].isna()&~df['fuel_level_dmm'].isna()]

### Eliminación de outliers

#### Filtrado de valores erróneos
A partir de la exploración de los datos definimos algunos criterios para el filtrado de nuestra base de datos. Por un lado, pudimos observar que la variable `volumen` posee valores mínimos negativos y que su máximo valor sobrepasa el valor máximo de la capacidad de los tanques, por lo que difinimos la primera y segunda condición de filtrado (cond_1 y cond_2) y nos quedamos con valores de volúmenes mayores o iguales a cero y menores a la capacidad máxima. Por otro lado, vimos que cuando la carga de alguna de las dos baterías es cero el nivel de combustible también es cero o produce valores nulos (nan) por lo que no se estaría estimando correctamente el valor de volumen, definiendo de esta manera las condidiones 3 y 4 (cond_3 y cond_4). Por último, no tuvimos en cuenta los valores nulos de código porque inferimos que el sensor no está funcionando y esto coincide con la existencia de valures nulos de temperatura y de nivel de combustible (cond_5).

In [ ]:
# Volúmenes negativos
cond_1 = df["volumen"] >= 0

# Volúmenes mayores a la capacidad del tanque
cond_2 = df["volumen"] < df["capacidad"].max()

# Voltaje del pulso de eco 0
cond_3 = df["vbat1"] != 0

# Batería con voltaje 0
cond_4 = df["vbat2"] != 0

# Esto lo pondría arriba, con los valores nulos
cond_5 = ~df['codigo'].isna()

In [ ]:
df = df[cond_1 & cond_2 & cond_3 & cond_4 & cond_5]
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().round()

In [ ]:
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize=(15,9),gridspec_kw={"height_ratios": (.15, .85)})
 
seaborn.boxplot(df.volumen, ax=ax_box)
seaborn.distplot(df.volumen, ax=ax_hist, kde=False, hist=False)
seaborn.distplot(df.volumen, ax=ax_hist, kde=True, hist=True,norm_hist=False)

plt.axvline(df.volumen.quantile(0.25))
plt.axvline(df.volumen.quantile(0.50),color='g')
plt.axvline(df.volumen.quantile(0.75))
plt.ticklabel_format(style='plain', axis='x') 
plt.axvline(df.volumen.mean(),color='red') 
ax_box.set_title
plt.show()

#### Filtrado teniendo en cuenta el rango intercuartílico

In [ ]:
q1 = df.volumen.quantile(0.25)
q3 = df.volumen.quantile(0.75)
RI = q3 - q1
min = q1 - 2.5*RI
max = q3 + 2.5*RI
print("Límite inferior =", min)
print("Límite superior =", max)

In [ ]:
df = df[df.volumen < max]
df.head()

#### Filtrado teniendo en cuenta el la desviación estándar

In [ ]:
vol_mean = df.volumen.mean()
vol_std = df.volumen.std()
min_s = vol_mean - 2.5 * vol_std
max_s = vol_mean + 2.5 * vol_std
print("Límite inferior =", min_s)
print("Límite superior =", max_s)

In [ ]:
df = df[df.volumen < max_s]
df.head()

In [ ]:
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize=(15,9),gridspec_kw={"height_ratios": (.15, .85)})
 
seaborn.boxplot(df.volumen, ax=ax_box)
seaborn.distplot(df.volumen, ax=ax_hist, kde=False, hist=False)
seaborn.distplot(df.volumen, ax=ax_hist, kde=True, hist=True,norm_hist=False)

plt.axvline(df.volumen.quantile(0.25))
plt.axvline(df.volumen.quantile(0.50),color='g')
plt.axvline(df.volumen.quantile(0.75))
plt.ticklabel_format(style='plain', axis='x') 
plt.axvline(df.volumen.mean(),color='red') 
ax_box.set_title
plt.show()

#### Filtrado de registros con códigos de error

Quitamos aquellos registros que contengan códigos de error. Para esto, separamos la columna `codigo` en dos campos, ya que la misma contiene además información de la cantidad de ecos.

Se deja la [documentación](#) sobre los significados de estos códigos.

In [ ]:
df['codigo'].unique()

In [ ]:
# Separamos el código en dos datos
df['c'] = df['codigo'].apply(lambda x: str(x)[0])
df['echoes'] = pd.to_numeric(df['codigo'].apply(lambda x: str(x)[1]), errors='coerce')

Veamos si quitar los valores con códigos de error obtenemos los mismos resultados que mantener los valores con códigos correctos.

In [ ]:
ok_codes = ['N', 'L', 'V', 'U', 'P', 'T', 'A', 'I', '0']
error_codes = ['m', 'M', 'F']

In [ ]:
# Obtenemos el mismo resultado?
print("Manteniendo los valores correctos:", len(df.query("c not in @ok_codes")))
print("Quitando los valores de error:", len(df.query("c in @error_codes")))

Obtenemos los mismo resultado, elegimos uno de ellos.

In [ ]:
df = df.query("c in @ok_codes")
len(df)

#### Filtrado de registros con ecos cero y ecos mayores a dos

La cantidad de ecos corresponde a la cantidad de señales recibidas en el transmisor.

Si el valor es cero, puede haber ocurrido dos cosas:
- Son sondas de presión, es decir, este tipo de sondas funcionan de manera distinta y la cantidad de ecos no corresponde.
- Los datos enviados son erróneos.

In [ ]:
df = df.query("echoes != 0")
len(df)

Los ecos mayores a dos son erróneos, ya que actualmente la cantidad de mediciones que se realizan son una o dos. Si reporta dos ecos, la sonda, además de medir el líquido principal (generalmente combustible), mide agua.

In [ ]:
df = df.query("echoes <= 2")
len(df)

#### Filtrado por pérdida de ecos en un mismo tanque
Cuando se pierden señales de eco en sondas que sólo miden un líquido, el código mostrará valores ceros durante esas pérdidas. Pero, ¿qué sucede si la sonda mide dos líquidos y pierde una de las señales? La sonda reportará un valor normal (uno), pero que resulta incorrecto para esta sonda.

No podemos solamente limitarnos a filtrar aquellos con ecos cero, debemos además quitar aquellos que debían ser dos y fue sólo una. Este análisis se hará por tanque.


#### Filtrado por rango intercuartílico en el tiempo
Calculamos el rango intercuartílico del `volumen`. https://anomaly.io/anomaly-detection-moving-median-decomposition/index.html

In [ ]:
window = 5
df['moving_median'] = df.groupby('id_tanque')['volumen'].transform(
    lambda x: x.rolling(window=5, min_periods=1, center=True).median()
)
df['q1'] = df.groupby('id_tanque')['volumen'].transform(
    lambda x: x.rolling(window=5, min_periods=1, center=True).quantile(0.25)
)
df['q3'] = df.groupby('id_tanque')['volumen'].transform(
    lambda x: x.rolling(window=5, min_periods=1, center=True).quantile(0.75)
)
df['RI'] = df['q3'] - df['q1']
df['RI_min'] = df['q1'] - 2.5*df['RI']
df['RI_max'] = df['q3'] + 2.5*df['RI']
df.tail()

In [ ]:
fig = plt.figure(figsize=(15, 7))
seaborn.lineplot(
    data=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900], 
    y='volumen', 
    x='timestamp' 
)
seaborn.lineplot(
    data=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900], 
    y='moving_median', 
    x='timestamp' 
)
seaborn.lineplot(
    data=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900], 
    y='RI_min', 
    x='timestamp' 
)
seaborn.lineplot(
    data=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900], 
    y='RI_max', 
    x='timestamp' 
)
plt.ticklabel_format(style='plain', axis='y')
seaborn.despine()

Quitamos entonces < RI * 2.5, > RI * 2.5

In [ ]:
# df = df.query("volumen < RI_min")
# df = df.query("volumen > RI_max")
# len(df)

## Análisis de correlación entre las variables numéricas

In [ ]:
df_cor=df_fil_out.corr(method="spearman")
df_cor

In [ ]:
plt.figure(figsize=(15,15))
seaborn.heatmap(df_cor, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'coolwarm')
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.show()

A partir del análisis de correlación entre las variables numéricas de nuestro dataset, pudimos observar una correlación alta entre las variables de temperatura (r > 0.7). Debido a que la feature temperatura representa los mismos valores que la feature temp5, sumado a que es la que indica la temperatura del combustible (a diferencia de las temp1,temp2, temp3 y temp4) decidimos quedarnos con esta variable y desechar las otras. Por otra parte, observamos un alta correlación entre volumen y nivel de combustible (r= 0.92), que es lo que se espera ya que la variable volumen se calcula a partir de la variable nivel de combustible. De hecho, esperaríamos obtener una correlación igual a uno, sin embargo esta diferencia en el nivel de correlación podría deberse a un error de cálculo a partir de la tabla de calibración. En este caso, nos quedaríamos con la variable volumen ya que es la que nos aporta la información necesaria para responder a las preguntas planteadas. Por último, existe una correlación alta entre volumen y capacidad (r = 0.82), lo cual es esperable, sin embargo la variable capacidad nos provee información que la variable valumen no nos da, por lo que decidimos mantenerla en nuestro dataset por el momento.

### Selección de columnas relevantes

In [ ]:
relevant_columns=[]

### Uniformidad de tiempo de los datos

A partir del siguiente análisis buscamos detectar gaps temporales entre muestras. El código presenta todos aquellos tanques (id_tanques) que presentan gaps entre muestras con más de 1 hora, junto a la media de los gaps y el desvío stándar. Oportunamente ajustaremos el gaps buscado en función de la frecuencia mínima requerida para la predección. 

In [ ]:
#se arma un array con los id_tanque
tanques_unique=df.id_tanque.unique()
for tanque in tanques_unique:
    # Tomar la diferencia del timestamp y no tener en cuenta la primer fila ya que no tiene diff hacia atrás
    deltas = df[df['id_tanque']==tanque].timestamp.diff()[1:]

    # Filtrar diffs mayores al valor introducido (pueden ser dias, horas, minutos, etc) 
    gaps = deltas[deltas > timedelta(hours=1)]

    #Resultados
    if len(gaps) != 0:
        print(f'Tanque ID:{tanque} tiene {len(gaps)} gaps con duración promedio: {gaps.mean()} y std: {gaps.std()}')
        #descomentar para ampliar información de los gaps
        #for i, g in gaps.iteritems():
        #    gap_start = df['timestamp'][i - 1]
        #    print(f'Start: {datetime.strftime(gap_start, "%Y-%m-%d")} | '
        #          f'Duration: {str(g.to_pytimedelta())}')

### Resampling del timeseries

Planteamos el resampleo de series de tiempo sobre distintas ventanas y distintos métodos. Se verá en mayor profundidad este tratamiento, en el práctico de curación.

In [ ]:
# 10 minutos
tanque_resampled = tanque.resample('10T').bfill()
tanque_resampled

In [ ]:
fig = plt.figure(figsize=(15, 7))
seaborn.lineplot(
    data=tanque_resampled[1400:1500], 
    y='volumen', 
    x='timestamp' 
)
plt.ticklabel_format(style='plain', axis='y')
seaborn.despine()

### Agrupación de inventarios

### Normalización de valores

Ideas: (esto es curación?) 
- Ajustar por temperatura.
- Restar volumen de agua.

## Análisis

Para esta etapa se procede a analizar los distintos inventarios agrupándolos por diferentes criterios.

### Descriptores estadísticos de los inventarios

### Invetarios sobre centros operativos

Se seleccionan algunos centros operativos para analizar el comportamiento de sus inventarios. Para esto, realizamos una selección aleatoria de la columna `id_equipo`.

### Patrones de manejo de inventario por industria

In [ ]:
df_ind=df[["volumen", "industria"]].groupby("industria").describe()
df_ind

In [ ]:
df_ind.columns = df_ind.columns.droplevel(level=0)
df_ind.sort_values(by='50%',ascending=False)

In [ ]:
plt.figure(figsize=(12,6))
seaborn.boxplot(data=df, x="industria", y="volumen")

In [ ]:
df_resa=df.resample("1H", on="timestamp").mean()
df_resa


### Patrones de manejo de inventario por producto


In [ ]:
df_pro=df_fil_out[["volumen", "nombre_producto"]].groupby("nombre_producto").describe()
df_pro

In [ ]:
df_pro.columns = df_pro.columns.droplevel(level=0)
df_pro.sort_values(by='50%',ascending=False)

In [ ]:
plt.figure(figsize=(12,6))#ver de agrupar productos, categorizaría primero porque sino el gráfico no dice nada (por ejemplo: diesel, gasoil, nafta y otros para los menos frecuentes)
seaborn.boxplot(data=df, x="nombre_producto", y="volumen")

In [ ]:
df_resa=df.resample("1H", on="timestamp").mean()
df_resa

### Primera y segunda derivada del `volumen` en el tiempo

La primer derivada nos da información de los movimientos del `volumen` en el tiempo. Si el líquido sube, la pendiente es positiva, si es negativa, el líquido baja. Valores cercanos a cero nos indican poco movimiento, mientras que valores más grandes nos indican que el líquido se mueve en grandes cantidades.

In [ ]:
# Cálculo de diferencia entre valores para obtener la pendiente
df['volumen_diff'] = df.groupby('id_tanque')['volumen'].diff()

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(15, 7))
seaborn.lineplot(
    data=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900], 
    y='volumen', 
    x='timestamp',
    ax=axes[0]
)
seaborn.lineplot(
    data=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900], 
    y='volumen_diff', 
    x='timestamp',
    ax=axes[1]
)
plt.ticklabel_format(style='plain', axis='y')
seaborn.despine()

La segunda derivada nos puede indicar cuál es la aceleración del movimiento del tanque.

In [ ]:
# Cálculo de diferencia entre diferencias para obtener la aceleración
df['volumen_second_diff'] = df.groupby('id_tanque')['volumen_diff'].diff()

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(15, 7))
seaborn.lineplot(
    data=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900], 
    y='volumen', 
    x='timestamp',
    ax=axes[0]
)
seaborn.lineplot(
    data=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900], 
    y='volumen_second_diff', 
    x='timestamp',
    ax=axes[1]
)
plt.ticklabel_format(style='plain', axis='y')
seaborn.despine()

### Consumos por unidad de tiempo por centro operativo

Se seleccionan algunos centros operativos.

Para poder calcular los consumos, debemos sumar todas las diferencias negativas por tanque. Intentemos sumar primero utilizando la columna calculada `volumen_diff`.

Elegimos distintos rangos de tiempo para calcular el consumo.

In [ ]:
df['period_hour'] = df['timestamp'].dt.to_period('H')

In [ ]:
# Nos quedamos con los valores negativos para facilitar la suma
df['neg_volumen_diff'] = numpy.where(
    df['volumen_diff'] < 0, 
    df['volumen_diff'].abs(), 
    0
)

In [ ]:
df['hourly_consumos'] = df.groupby('id_tanque')['neg_volumen_diff'].sum()

In [ ]:
fig = plt.figure(figsize=(15, 7))
seaborn.boxplot(
    y=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900]['hourly_consumos'], 
    x=df[df['id_tanque'] == df['id_tanque'].cat.categories[0]][24000:33900]['period_hour']
)
plt.ticklabel_format(style='plain', axis='y')
seaborn.despine()

A veces puede haber movimientos en los tanques, por lo tanto, no sería correcto utilizar las diferencias entre puntos sin ningún tratamiento. Necesitamos suavizar los registros para obtener las diferencias.

In [ ]:
df['moving_avg'] = df.groupby('id_tanque')['volumen'].transform(
    lambda x: x.rolling(window=5, min_periods=1, center=True).mean()
)

In [ ]:
df['volumen_diff'] = df.groupby('id_tanque')['moving_avg'].diff()

### Patrones de consumo por industria

### Análisis estadísitico de los consumos en general

### Análisis estadístico de los consumos por industria


Ver si los patrones de consumo de minería y telecomunicaciones son diferentes. Deberíamos explica por qué creemos encontrar diferencia entre estas dos industrias? Quizas para minería se necesite más combustible ya que se utilizan grandes maquinarias que insumen más combustible que las maquinarias que se emplean en telecomunicaciones. Igual con distintos productos, por ejemplo:  gas y nafta, el gas es más barato que la nafta por lo que podríamos esperar un mayor consumo de este 

### Media móvil de inventarios para cada indsutria

Se selecciona un centro operativo por industria para realizar el análisis.

In [ ]:
# Ventana numérica

In [ ]:
# Ventana por tiempo

### Media móvil de consumos para cada industria

In [ ]:
# Ventana numérica

In [ ]:
# Ventana de tiempo

## Conclusiones

Cómo harían un cálculo simple para estimar inventarios en el corto plazo?

Ideas:
- Repetir Último valor visto.
- Drift (media de últimos valores vistos)
- Última pendiente
- Media movil de últimas pendientes
- última pendiente, y aplicar última aceleración (d' y d'')
- Media movil de últimas pendientes y aplicar mm de última aceleración
- Conjunto de últimos valores + último valor 

### Repetición del último valor visto en el tiempo.

### Último valor visto y media móvil

## Notas

Ordenar:  
Limpieza
1. Valores faltantes
 - Ver volúmenes nulos si hay
 
2. Outliers
  - Volúmenes negativos (quitar)
  - Volúmenes por encima de la capacidad 
  - Volúmenes no numéricos
  - Tanques con pocos registros
  - Volúmenes que se salen de cierto rango alrededor de la media móvil (intervalos de confianza? porcentajes? ver histogramas?)
  - Registros con códigos de error (definidas en tabla)
  - Registros con ecos 0 (tener en cuenta sondas de presión)
  - Temp5 fuera de rango (idem anterior media movil? intervalos?)
  - vbat1/2 fuera de rango (idem anterior)
  - Capacidades negativas, muy chicas. Ver histograma

3. No se encuentran uniformes
  - Graficar un par donde se vean
  - Calcular diff de tiempos, distribucion.

4. Resample
  - interpolación (lineal? cúbica?)
  - con bfill, etc, mostrar que no está bueno.
  - Interpolación en espacios de tiempo muy largos.

5. Agrupación
  - Tanques

6. Normalizar valores
  - Variación por temperatura?
  - Variación por cantidad de agua?

Análisis
1. Gráfico de centros op seleccionados.
2. Patrones por industria (análisis que hizo Lau)
3. Descriptores estadísticos
  - Medidas de centralización
  - Medidas de dispersión
4. Manejo de inventarios por industria (idem 2?) (Lau) 
5. Correlación de features (analisis de Lau)
6. Derivada primera y derivada segunda
  - Derivada primera nos dice si el líquido sube o baja.
  - Derivada segunda, velocidad de crecimiento/decr? aceleración de crec/decr?
7. Consumo por unidad de tiempo por centro operativo.
  - Suma de diffs negativos absoluto? Quizás sobre media movil. Ver dispersión de media movil
8. Patrones de consumo por industria.
9. Descriptores estadísticos consumo general y por industria.
10. Media móvil de centros operativos por industria.
11. Media móvil de consumos de centros operativos por industria.



===================================

Análisis:
- Autocorrelación (correlacion para series temporales sobre los lags)

Plots:
- Time plot
- Seasonal plot
- Polar plot
- Lag plot (scatter?)

Predicciones naïve:
